In [1]:
import pandas as pd
import numpy as np
# from nltk.stem import WordNetLemmatizer
# import nltk
# nltk.download('omw-1.4')

import json
import random
random.seed(42)

In [2]:
data = pd.read_csv('data.zip')
data

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Positive_Review_clean,Negative_Review_clean,Negative_Review_lemmatized,Positive_Review_lemmatized
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,...,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,park outside hotel beautiful,angry made post available via possible sites u...,angry made post available via possible site us...,park outside hotel beautiful
1,1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Ireland,NaN,0,1403,...,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,real complaints hotel great great location sur...,NaN,NaN,real complaint hotel great great location surr...
2,2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,...,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968,location good staff ok cute hotel breakfast ra...,rooms nice elderly bit difficult rooms two sto...,room nice elderly bit difficult room two story...,location good staff ok cute hotel breakfast ra...
3,3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,...,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968,great location nice surroundings bar restauran...,room dirty afraid walk barefoot floor looked c...,room dirty afraid walk barefoot floor looked c...,great location nice surroundings bar restauran...
4,4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-24,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,...,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968,amazing location building romantic setting,booked company line showed pictures room thoug...,booked company line showed picture room though...,amazing location building romantic setting
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515733,515733,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,168,2015-08-30,8.1,Atlantis Hotel Vienna,Kuwait,no trolly or staff to help you take the lugga...,14,2823,...,8,7.0,"[' Leisure trip ', ' Family with older childre...",704 day,48.203745,16.335677,location,trolly staff help take luggage room,trolly staff help take luggage room,location
515734,515734,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,168,2015-08-22,8.1,Atlantis Hotel Vienna,Estonia,The hotel looks like 3 but surely not 4,11,2823,...,12,5.8,"[' Leisure trip ', ' Family with young childre...",712 day,48.203745,16.335677,breakfast ok got earlier check,hotel looks like 3 surely 4,hotel look like 3 surely 4,breakfast ok got earlier check
515735,515735,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,168,2015-08-19,8.1,Atlantis Hotel Vienna,Egypt,The ac was useless It was a hot week in vienn...,19,2823,...,3,2.5,"[' Leisure trip ', ' Family with older childre...",715 day,48.203745,16.335677,NaN,ac useless hot week vienna gave hot air,ac useless hot week vienna gave hot air,NaN
515736,515736,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,168,2015-08-17,8.1,Atlantis Hotel Vienna,Mexico,NaN,0,2823,...,3,8.8,"[' Leisure trip ', ' Group ', ' Standard Tripl...",717 day,48.203745,16.335677,rooms enormous really comfortable believe fami...,NaN,NaN,room enormous really comfortable believe famil...


In [4]:
hotel_list = list(data['Hotel_Name'].unique())
hotel_list = random.sample(hotel_list, 500)
len(hotel_list)

500

In [5]:
with open('docs/json_eu_map/hotel_data.geojson', 'r') as f:
    data_dict = json.load(f)

In [6]:
# each item in the list data_dict['features'] corresponds to a hotel
data_dict['features'][0]['properties']

{'name': 'Hotel Arena',
 'reviews': [{'rating': 2.9,
   'neg_review_words': ['angry',
    'made',
    'post',
    'available',
    'via',
    'possible',
    'site',
    'use',
    'planing',
    'trip',
    'one',
    'make',
    'mistake',
    'booking',
    'place',
    'made',
    'booking',
    'via',
    'booking',
    'com',
    'stayed',
    '6',
    'night',
    'hotel',
    '11',
    '17',
    'july',
    'upon',
    'arrival',
    'placed',
    'small',
    'room',
    '2nd',
    'floor',
    'hotel',
    'turned',
    'room',
    'booked',
    'specially',
    'reserved',
    '2',
    'level',
    'duplex',
    'room',
    'would',
    'big',
    'window',
    'high',
    'ceiling',
    'room',
    'ok',
    'mind',
    'broken',
    'window',
    'closed',
    'hello',
    'rain',
    'mini',
    'fridge',
    'contained',
    'sort',
    'bio',
    'weapon',
    'least',
    'guessed',
    'smell',
    'intimately',
    'asked',
    'change',
    'room',
    'explaining',

In [ ]:
for hotel in data_dict['features']:
    #print(hotel)
    if hotel['properties']['name'] not in hotel_list:
        data_dict['features'].remove(hotel)

In [13]:
len(data_dict['features'])

523

In [14]:
scoremap = { name: score for name, score in data[['Hotel_Name', 'Average_Score']].to_numpy() }

# so to add metadata to each hotel:
for hotel in data_dict['features']:
    hotel['properties']['average_score'] = scoremap[hotel['properties']['name']]

In [15]:
reviews = {name: [] for name in scoremap}
for row in data[['Hotel_Name', 'Reviewer_Score', 'Negative_Review_lemmatized', 'Positive_Review_lemmatized', 'Review_Date']].to_numpy():
    row2 = row[2]
    row3 = row[3]
    if type(row2) != str: row2 = ''
    if type(row3) != str: row3 = ''

    reviews[row[0]].append({'rating':row[1], 'neg_review_words': row2.split(' '), 'pos_review_words': row3.split(' '), 'date': row[4]})

In [16]:
for hotel in data_dict['features']:
    hotel['properties']['reviews'] = reviews[hotel['properties']['name']]

In [17]:
with open('docs/json_eu_map/hotel_data.geojson', 'w') as f:
    f.write(json.dumps(data_dict))